In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input, GlobalAveragePooling2D, Lambda
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

2025-02-16 16:43:59.578833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-16 16:44:02.430178: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6/lib64
2025-02-16 16:44:02.430416: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7

In [2]:
# Load the preprocessed data
X = np.load("/home/natalyagrokh/imgs_preprocessed/preproc_affectnet/X_affectnet.npy")
y = np.load("/home/natalyagrokh/imgs_preprocessed/preproc_affectnet/y_affectnet.npy")

In [3]:
# Reshape the data (if needed) to add a grayscale channel
if len(X.shape) == 3:
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# One-hot encode the labels
num_classes = len(np.unique(y))  # Determine the number of unique classes
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

In [6]:
# Calculate class weights to handle imbalanced datasets
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y),
    y=y
)
class_weights = dict(enumerate(class_weights))

In [7]:
   
# Define EfficientNetB0 Model
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

2025-02-16 16:44:13.267855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-16 16:44:13.682719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-16 16:44:13.688067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-16 16:44:13.692801: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [8]:
# Define callbacks
callbacks = [
    ModelCheckpoint(
        "final_trained_model_weights.h5", 
        monitor="val_accuracy", 
        save_best_only=True, 
        mode="max", 
        save_weights_only=True
    ),
    EarlyStopping(monitor="val_loss", patience=5, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=3, verbose=1)
]

In [9]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Adjust epochs as needed
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks
)

Epoch 1/20


2025-02-16 16:44:36.556048: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2025-02-16 16:44:39.465769: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2025-02-16 16:44:40.390559: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7efd4407cd80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-02-16 16:44:40.390595: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-02-16 16:44:40.445847: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-02-16 16:44:40.912471: I tensorflow/compiler/jit/xla_compilation_cache.cc

776/776 [==============================] - 72s 42ms/step - loss: 1.6504 - accuracy: 0.3727 - val_loss: 2.4652 - val_accuracy: 0.1268 - lr: 0.0010
Epoch 2/20
776/776 [==============================] - 32s 41ms/step - loss: 1.3384 - accuracy: 0.5165 - val_loss: 3.0159 - val_accuracy: 0.0922 - lr: 0.0010
Epoch 3/20
776/776 [==============================] - 29s 38ms/step - loss: 1.2308 - accuracy: 0.5667 - val_loss: 2.8913 - val_accuracy: 0.2109 - lr: 0.0010
Epoch 4/20
776/776 [==============================] - ETA: 0s - loss: 1.1557 - accuracy: 0.5998 
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
776/776 [==============================] - 32s 41ms/step - loss: 1.1557 - accuracy: 0.5998 - val_loss: 2.9064 - val_accuracy: 0.1229 - lr: 0.0010
Epoch 5/20
776/776 [==============================] - 29s 37ms/step - loss: 0.9885 - accuracy: 0.6594 - val_loss: 4.6908 - val_accuracy: 0.1272 - lr: 1.0000e-04
Epoch 6/20
776/776 [==============================] - 31s 4

In [10]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Accuracy with EfficientNetB0: {val_accuracy * 100:.2f}%")

194/194 [==============================] - 4s 18ms/step - loss: 4.1774 - accuracy: 0.1464
Validation Accuracy with EfficientNetB0: 14.64%


In [21]:
import tensorflow as tf

# Save the entire model using the lower-level API.
tf.saved_model.save(model, "final_efficientnet_trained_model")
print("Full model saved successfully as a SavedModel in 'final_efficientnet_trained_model'.")

INFO:tensorflow:Assets written to: final_efficientnet_trained_model/assets


INFO:tensorflow:Assets written to: final_efficientnet_trained_model/assets


Full model saved successfully as a SavedModel in 'final_efficientnet_trained_model'.


In [24]:
import json

# Define a custom encoder to handle EagerTensors
def custom_encoder(o):
    # If the object is an EagerTensor, convert it to a list.
    if hasattr(o, "numpy"):
        return o.numpy().tolist()
    # Otherwise, use the default string representation.
    raise TypeError(f"Object of type {type(o)} is not JSON serializable")

# Get the model configuration dictionary.
model_config = model._updated_config()

# Serialize the configuration with the custom encoder.
model_json = json.dumps(model_config, default=custom_encoder)

# Save the architecture to a JSON file.
arch_path = "/home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_architecture.json"
with open(arch_path, "w") as json_file:
    json_file.write(model_json)
print(f"Model architecture saved to JSON at {arch_path}.")

# Save the model weights to an H5 file.
weights_path = "/home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_weights.h5"
model.save_weights(weights_path)
print(f"Model weights saved to H5 at {weights_path}.")

Model architecture saved to JSON at /home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_architecture.json.
Model weights saved to H5 at /home/natalyagrokh/img_expressions/final_efficientnet_trained_model_2_weights.h5.


In [11]:
# # doesn't work -> throws a type error
# model.save("final_efficientnet_trained_model.h5", save_format="h5")
# print("Keras model saved successfully as 'final_efficientnet_trained_model'.")